In [1]:
import sqlite3
import os

pdf_db_path = "app_storage/metadata/pdf_documents.db"


def init_pdf_database():
    """Creates the SQLite database for PDF metadata if it doesn’t already exist."""
    if not os.path.exists(pdf_db_path):
        connection = sqlite3.connect(pdf_db_path)
        cursor = connection.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS pdf_metadata (
                id INTEGER PRIMARY KEY,
                file_name TEXT,
                file_path TEXT,
                title TEXT,
                author TEXT,
                creation_date TEXT,
                subject TEXT
            )
        ''')
        connection.commit()
        connection.close()


def insert_pdf_metadata(file_name, file_path, metadata):
    """Inserts metadata for a PDF into the database.
    Accepts file_name, file_path, and a dictionary of metadata fields."""
    connection = sqlite3.connect(pdf_db_path)
    cursor = connection.cursor()
    cursor.execute('''
        INSERT INTO pdf_metadata (file_name, file_path, title, author, creation_date, subject)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (file_name, file_path, metadata.get('title'), metadata.get('author'),
          metadata.get('creation_date'), metadata.get('subject')))
    connection.commit()
    connection.close()

In [2]:
pdf_db_path = "app_storage/metadata/pdf_documents.db"

In [3]:
import sqlite3
import sqlite_vec

db = sqlite3.connect('app_storage/metadata/pdf_documents.db')
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)
# 
# vec_version, = db.execute("select vec_version()").fetchone()
# print(f"vec_version={vec_version}")

OperationalError: unable to open database file

In [9]:
file_path = '/Users/aapoliakova/PycharmProjects/RAG_app_mozzilla/app_storage/pdfs/test1.pdf'
metadata = {'file_path': file_path, 'file_name': "some test file name", "author": "some author", "subject": "some subject"}
extracted_pdf = [
(1, "text of chunk one"),
(2, "text of chunk two"),
(3, "text of chunk three"),
(4, "text of chunk four")
]



embeddings = [
    (1, [0.1, 0.1, 0.1, 0.1]),
    (2, [0.2, 0.2, 0.2, 0.2]),
    (3, [0.3, 0.3, 0.3, 0.3]),
    (4, [0.4, 0.4, 0.4, 0.4]),
]

In [16]:
def create_one_document_entries(metadata, extracted_pdf, embeddings):
    entries = []
    for (i, chunk), (i, embedding) in zip(extracted_pdf, embeddings):
        entries.append((i, chunk, embedding, metadata['file_path'], metadata['file_name'], metadata['author'], metadata['subject']))
        return entries
    
entries = create_one_document_entries(metadata, extracted_pdf, embeddings)
entries[0]

(1,
 'text of chunk one',
 [0.1, 0.1, 0.1, 0.1],
 '/Users/aapoliakova/PycharmProjects/RAG_app_mozzilla/app_storage/pdfs/test1.pdf',
 'some test file name',
 'some author',
 'some subject')

In [22]:
import sqlite3
import sqlite_vec

db = sqlite3.connect("tmp_database")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)


db.execute("CREATE VIRTUAL TABLE vec_items USING vec0(embedding float[4])")

In [16]:
import sqlite3
import sqlite_vec

from typing import List
import struct

db.execute("DROP TABLE IF EXISTS vec_items")
# db.execute("DROP TABLE IF EXISTS metadata")
def serialize_f32(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)


db = sqlite3.connect("tmp_database")
db.enable_load_extension(True)
sqlite_vec.load(db)


sqlite_version, vec_version = db.execute(
    "select sqlite_version(), vec_version()"
).fetchone()
print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")

items = [
    (1, 1, [0.1, 0.1, 0.1, 0.1]),
    (2, 2, [0.2, 0.2, 0.2, 0.2]),
    (3, 2, [0.3, 0.3, 0.3, 0.3]),
    (4, 3, [0.4, 0.4, 0.4, 0.4]),
]
query = [0.3, 0.3, 0.3, 0.3]

db.execute("""CREATE VIRTUAL TABLE vec_items USING vec0(
    article_id primary key, 
    document_id primary key,
    embedding float[4])""")
# create virtual table vec_articles using vec0(
#   article_id integer primary key,
#   headline_embedding float[768]
# );
with db:
    for item in items:
        db.execute(
            "INSERT INTO vec_items(rowid, id, embedding) VALUES (?, ?, ?)",
            [item[0], item[1], serialize_f32(item[2])],
        )

rows = db.execute(
    """
      SELECT
        rowid,
        distance
      FROM vec_items
      WHERE embedding MATCH ?
      ORDER BY distance
      LIMIT 3
    """,
    [serialize_f32(query)],
).fetchall()

print(rows)

sqlite_version=3.45.3, vec_version=v0.1.3


OperationalError: vec0 constructor error: Unknown table option: article_id

In [6]:
db.execute('SELECT * FROM pdf_metadata')

OperationalError: no such table: pdf_metadata

In [25]:
import sqlite3
import sqlite_vec
from typing import List
import struct


def serialize_f32(vector: List[float]) -> bytes:
    """Serializes a list of floats into a compact 'raw bytes' format"""
    return struct.pack("%sf" % len(vector), *vector)


# Connect to the SQLite database and load the sqlite_vec extension
db = sqlite3.connect("extended_database.db")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

# Check SQLite and Vec versions
sqlite_version, vec_version = db.execute(
    "SELECT sqlite_version(), vec_version()"
).fetchone()
print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")
# db.execute("CREATE VIRTUAL TABLE vec_items USING vec0(embedding float[4])")
# Create a virtual table with additional columns for metadata
db.execute("""
    CREATE VIRTUAL TABLE vec_items USING vec0(embedding float[4])
        chunk_id INTEGER PRIMARY KEY,  
        chunk_text TEXT,  
        file_path TEXT,  
        file_name TEXT,  
        author TEXT,  
        abstract TEXT 
    )
""")

# Sample data entries
items = [
    (1, [0.1, 0.1, 0.1, 0.1], "Chunk text 1", "/path/to/file1", "file1.pdf", "Author A", "Abstract A"),
    (2, [0.2, 0.2, 0.2, 0.2], "Chunk text 2", "/path/to/file1", "file1.pdf", "Author A", "Abstract A"),
    (3, [0.3, 0.3, 0.3, 0.3], "Chunk text 3", "/path/to/file2", "file2.pdf", "Author B", "Abstract B"),
    (4, [0.4, 0.4, 0.4, 0.4], "Chunk text 4", "/path/to/file3", "file3.pdf", "Author C", "Abstract C"),
]
query_embedding = [0.3, 0.3, 0.3, 0.3]  # Query vector for similarity search

# Insert entries into the database
with db:
    for item in items:
        db.execute(
            """
            INSERT INTO vec_items(
                chunk_id, embedding, chunk_text, file_path, file_name, author, abstract
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """,
            [item[0], serialize_f32(item[1]), item[2], item[3], item[4], item[5], item[6]]
        )

# Perform a similarity search with the query vector
rows = db.execute(
    """
    SELECT
        chunk_id,
        chunk_text,
        file_path,
        file_name,
        author,
        abstract,
        distance
    FROM vec_items
    WHERE embedding MATCH ?
    ORDER BY distance
    LIMIT 3
    """,
    [serialize_f32(query_embedding)]
).fetchall()

# Print query results
for row in rows:
    print(f"chunk_id: {row[0]}, chunk_text: {row[1]}, file_path: {row[2]}, "
          f"file_name: {row[3]}, author: {row[4]}, abstract: {row[5]}, distance: {row[6]}")


sqlite_version=3.45.3, vec_version=v0.1.3


OperationalError: near "chunk_id": syntax error

In [6]:
import sqlite3
import sqlite_vec
from typing import List
import struct


def serialize_f32(vector: List[float]) -> bytes:
    """Serializes a list of floats into a compact 'raw bytes' format"""
    return struct.pack("%sf" % len(vector), *vector)


# Connect to the SQLite database and load the sqlite_vec extension
db = sqlite3.connect("extended_database.db")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

# Check SQLite and Vec versions
sqlite_version, vec_version = db.execute(
    "SELECT sqlite_version(), vec_version()"
).fetchone()
print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")

# Create a vector-only virtual table for embeddings
db.execute("CREATE VIRTUAL TABLE vec_items USING vec0(embedding float[4])")

# Create a separate table for metadata, linked by chunk_id
db.execute("""
    CREATE TABLE metadata (
        chunk_id INTEGER PRIMARY KEY,  -- Unique identifier for the chunk
        chunk_text TEXT,  -- Text of the chunk
        file_path TEXT,  -- Path to the file
        file_name TEXT,  -- Name of the file
        author TEXT,  -- Author of the document
        abstract TEXT  -- Abstract of the document
    )
""")

# Sample data entries
items = [
    (1, [0.1, 0.1, 0.1, 0.1], "Chunk text 1", "/path/to/file1", "file1.pdf", "Author A", "Abstract A"),
    (2, [0.2, 0.2, 0.2, 0.2], "Chunk text 2", "/path/to/file2", "file2.pdf", "Author B", "Abstract B"),
    (3, [0.3, 0.3, 0.3, 0.3], "Chunk text 3", "/path/to/file3", "file3.pdf", "Author C", "Abstract C"),
    (4, [0.4, 0.4, 0.4, 0.4], "Chunk text 4", "/path/to/file4", "file4.pdf", "Author D", "Abstract D"),
]
query_embedding = [0.3, 0.3, 0.3, 0.3]  # Query vector for similarity search

# Insert embeddings into vec_items and metadata into metadata table
with db:
    for item in items:
        # Insert the embedding into vec_items
        db.execute(
            "INSERT INTO vec_items(rowid, embedding) VALUES (?, ?)",
            [item[0], serialize_f32(item[1])]
        )
        # Insert metadata into the metadata table
        db.execute(
            """
            INSERT INTO metadata(
                chunk_id, chunk_text, file_path, file_name, author, abstract
            ) VALUES (?, ?, ?, ?, ?, ?)
            """,
            [item[0], item[2], item[3], item[4], item[5], item[6]]
        )

# Perform a similarity search with the query vector and join with metadata
rows = db.execute(
    """
    SELECT
        m.chunk_id,
        m.chunk_text,
        m.file_path,
        m.file_name,
        m.author,
        m.abstract,
        v.distance
    FROM vec_items v
    JOIN metadata m ON v.rowid = m.chunk_id
    WHERE v.embedding MATCH ?
    ORDER BY v.distance
    LIMIT 3
    """,
    [serialize_f32(query_embedding)]
).fetchall()

# Print query results
for row in rows:
    print(f"chunk_id: {row[0]}, chunk_text: {row[1]}, file_path: {row[2]}, "
          f"file_name: {row[3]}, author: {row[4]}, abstract: {row[5]}, distance: {row[6]}")


sqlite_version=3.45.3, vec_version=v0.1.3


OperationalError: A LIMIT or 'k = ?' constraint is required on vec0 knn queries.

In [5]:
db.execute("DROP TABLE IF EXISTS vec_items")
db.execute("DROP TABLE IF EXISTS metadata")